In [1]:
import os
import pandas as pd
import numpy as np
import json
import sys
sys.path.append("..")
from algorithm import a_star_algorithm
from arm_state import ArmState, set_arm_length
from hold import Hold
from human_state import HumanState, set_leg_spread
from leg_state import LegState, set_leg_length
from utils import Side
from search_node import Search_Node
from visualization import create_animation

In [2]:
def create_json(experiment_name, path):
    path_json = []
    for node in path:
        human_state = {
            "left_arm": {
                "hand_position": node.human_state.left_arm_state.hand_position.tolist(),
                "shoulder_position": node.human_state.left_arm_state.shoulder_position.tolist(),
            },
            "right_arm": {
                "hand_position": node.human_state.right_arm_state.hand_position.tolist(),
                "shoulder_position": node.human_state.right_arm_state.shoulder_position.tolist(),
            },
            "left_leg": {
                "foot_position": node.human_state.left_leg_state.foot_position.tolist(),
                "knee_position": node.human_state.left_leg_state.knee_position.tolist(),
                "hip_position": node.human_state.left_leg_state.hip_position.tolist(),
            },
            "right_leg": {
                "foot_position": node.human_state.right_leg_state.foot_position.tolist(),
                "knee_position": node.human_state.right_leg_state.knee_position.tolist(),
                "hip_position": node.human_state.right_leg_state.hip_position.tolist(),
            }
        }
        path_json.append(human_state)

    # Convert to JSON string (you can also write this to a file if needed)
    path_json_str = json.dumps(path_json, indent=2)
    
    file_path = f"results\{experiment_name}_climber_path.json"
    # save the path to a json file
    with open(file_path, "w") as file:
        file.write(path_json_str)
    print(f"Path has been saved to {file_path}.")

# Same wall, different climbers

### Wall

In [20]:
# Define wall holds and goal state positions.
base_wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 11, 2) for y in range(1, 21, 2)
    ]

goal_state_positions = [np.array([9,19]), np.array([9,19])]

### Climber 1

In [4]:
# Define starting left leg configuration.
starting_left_foot_hold = Hold(3, 1)
starting_left_knee_position = np.array([2,2.75])
starting_left_hip_position = np.array([3, 4])

# Define starting right leg configuration.
starting_right_foot_hold = Hold(5, 1)
starting_right_knee_position = np.array([6, 2.75])
starting_right_hip_position = np.array([5, 4])

# Define starting left arm configuration.
starting_left_hand_hold = Hold(1, 7)
starting_left_shoulder_position = np.array([3, 5.5])

# Define starting right arm configuration.
starting_right_hand_hold = Hold(5,7)
starting_right_shoulder_position = np.array([5, 5.5])

# Define initial human state based on starting body configurations.
starting_left_leg_state = LegState(Side.LEFT, starting_left_foot_hold, starting_left_knee_position,
                                    starting_left_hip_position)
starting_right_leg_state = LegState(Side.RIGHT, starting_right_foot_hold, starting_right_knee_position,
                                    starting_right_hip_position)
starting_left_arm_state = ArmState(Side.LEFT, starting_left_hand_hold, starting_left_shoulder_position)
starting_right_arm_state = ArmState(Side.RIGHT, starting_right_hand_hold, starting_right_shoulder_position)
starting_human_state = HumanState(starting_left_leg_state, starting_right_leg_state, starting_left_arm_state,
                                    starting_right_arm_state)

# Define body physique constants.
set_leg_spread(np.float_(np.float64(2.0943951)))
set_arm_length(np.float_(np.float64(2.5)))
set_leg_length(starting_right_foot_hold, starting_right_knee_position, starting_right_hip_position)

In [5]:
path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, base_wall_holds)

In [20]:
create_json("2climbers_samewall", path)

Path has been saved to results\2climbers_samewall_climber_path.json.


### Climber 2 (longer arms)

In [26]:
# Define starting left leg configuration.
starting_left_foot_hold = Hold(3, 1)
starting_left_knee_position = np.array([2,2.75])
starting_left_hip_position = np.array([3, 4])

# Define starting right leg configuration.
starting_right_foot_hold = Hold(5, 1)
starting_right_knee_position = np.array([6, 2.75])
starting_right_hip_position = np.array([5, 4])

# Define starting left arm configuration.
starting_left_hand_hold = Hold(1, 7)
starting_left_shoulder_position = np.array([3, 5.5])

# Define starting right arm configuration.
starting_right_hand_hold = Hold(5,7)
starting_right_shoulder_position = np.array([5, 5.5])

# Define initial human state based on starting body configurations.
starting_left_leg_state = LegState(Side.LEFT, starting_left_foot_hold, starting_left_knee_position,
                                    starting_left_hip_position)
starting_right_leg_state = LegState(Side.RIGHT, starting_right_foot_hold, starting_right_knee_position,
                                    starting_right_hip_position)
starting_left_arm_state = ArmState(Side.LEFT, starting_left_hand_hold, starting_left_shoulder_position)
starting_right_arm_state = ArmState(Side.RIGHT, starting_right_hand_hold, starting_right_shoulder_position)
starting_human_state = HumanState(starting_left_leg_state, starting_right_leg_state, starting_left_arm_state,
                                    starting_right_arm_state)

# Define body physique constants.
set_leg_spread(np.float_(np.float64(2.0943951)))
set_arm_length(np.float_(np.float64(3)))
set_leg_length(starting_right_foot_hold, starting_right_knee_position, starting_right_hip_position)

In [27]:
path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, base_wall_holds)

In [28]:
create_json("2climbers_samewall(taller)", path)

Path has been saved to results\2climbers_samewall(taller)_climber_path.json.


### compare

In [44]:
def compare_jsons(json1_path, json2_path):
    with open(json1_path, "r") as file:
        json1 = json.load(file)
    with open(json2_path, "r") as file:
        json2 = json.load(file)
    if len(json1) != len(json2):
        print(f"The paths have different lengths. \n number of steps in {json1_path}: {len(json1)} \n number of steps in {json2_path}: {len(json2)}")
        return
    print("The paths are the same.")

In [53]:
file1_path = 'results\\2climbers_samewall_climber_path.json'
file2_path = 'results\\2climbers_samewall(taller)_climber_path.json'
compare_jsons(file1_path, file2_path)

The paths have different lengths. 
 number of steps in results\2climbers_samewall_climber_path.json: 17 
 number of steps in results\2climbers_samewall(taller)_climber_path.json: 16


# Same climber, different walls

### Climber 1

In [48]:
# Define starting left leg configuration.
starting_left_foot_hold = Hold(3, 1)
starting_left_knee_position = np.array([2,2.75])
starting_left_hip_position = np.array([3, 4])

# Define starting right leg configuration.
starting_right_foot_hold = Hold(5, 1)
starting_right_knee_position = np.array([6, 2.75])
starting_right_hip_position = np.array([5, 4])

# Define starting left arm configuration.
starting_left_hand_hold = Hold(1, 7)
starting_left_shoulder_position = np.array([3, 5.5])

# Define starting right arm configuration.
starting_right_hand_hold = Hold(5,7)
starting_right_shoulder_position = np.array([5, 5.5])

# Define initial human state based on starting body configurations.
starting_left_leg_state = LegState(Side.LEFT, starting_left_foot_hold, starting_left_knee_position,
                                    starting_left_hip_position)
starting_right_leg_state = LegState(Side.RIGHT, starting_right_foot_hold, starting_right_knee_position,
                                    starting_right_hip_position)
starting_left_arm_state = ArmState(Side.LEFT, starting_left_hand_hold, starting_left_shoulder_position)
starting_right_arm_state = ArmState(Side.RIGHT, starting_right_hand_hold, starting_right_shoulder_position)
starting_human_state = HumanState(starting_left_leg_state, starting_right_leg_state, starting_left_arm_state,
                                    starting_right_arm_state)

# Define body physique constants.
set_leg_spread(np.float_(np.float64(2.0943951)))
set_arm_length(np.float_(np.float64(2.5)))
set_leg_length(starting_right_foot_hold, starting_right_knee_position, starting_right_hip_position)

### Climber 2

In [3]:
# Define starting left leg configuration.
starting_left_foot_hold = Hold(3, 1)
starting_left_knee_position = np.array([2,2.75])
starting_left_hip_position = np.array([3, 4])

# Define starting right leg configuration.
starting_right_foot_hold = Hold(5, 1)
starting_right_knee_position = np.array([6, 2.75])
starting_right_hip_position = np.array([5, 4])

# Define starting left arm configuration.
starting_left_hand_hold = Hold(1, 7)
starting_left_shoulder_position = np.array([3, 5.5])

# Define starting right arm configuration.
starting_right_hand_hold = Hold(5,7)
starting_right_shoulder_position = np.array([5, 5.5])

# Define initial human state based on starting body configurations.
starting_left_leg_state = LegState(Side.LEFT, starting_left_foot_hold, starting_left_knee_position,
                                    starting_left_hip_position)
starting_right_leg_state = LegState(Side.RIGHT, starting_right_foot_hold, starting_right_knee_position,
                                    starting_right_hip_position)
starting_left_arm_state = ArmState(Side.LEFT, starting_left_hand_hold, starting_left_shoulder_position)
starting_right_arm_state = ArmState(Side.RIGHT, starting_right_hand_hold, starting_right_shoulder_position)
starting_human_state = HumanState(starting_left_leg_state, starting_right_leg_state, starting_left_arm_state,
                                    starting_right_arm_state)

# Define body physique constants.
set_leg_spread(np.float_(np.float64(2.0943951)))
set_arm_length(np.float_(np.float64(3)))
set_leg_length(starting_right_foot_hold, starting_right_knee_position, starting_right_hip_position)

### Wall 1 (hard)

In [4]:
# Define wall holds and goal state positions.
hard_wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 10, 3) for y in range(1, 20, 3)
    ]
hard_wall_holds.append(Hold(3, 1))
hard_wall_holds.append(Hold(5, 1))
hard_wall_holds.append(Hold(5, 7))
hard_wall_holds.append(Hold(9, 19))

goal_state_positions = [np.array([9,19]), np.array([9,19])]

In [6]:
path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, hard_wall_holds)

In [ ]:
create_json("sameclimber_hardwall", path)

Path has been saved to results\sameclimber_mediumwall_climber_path.json.


In [7]:
create_json("sameclimber2_hardwall", path)

Path has been saved to results\sameclimber2_hardwall_climber_path.json.


### Wall 2 (medium)

In [8]:
# Define wall holds and goal state positions.
medium_wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 10, 2) for y in range(1, 20, 3)
    ]
medium_wall_holds.append(Hold(3, 1))
medium_wall_holds.append(Hold(5, 1))
medium_wall_holds.append(Hold(5, 7))
medium_wall_holds.append(Hold(9, 19))

goal_state_positions = [np.array([9,19]), np.array([9,19])]

In [9]:
path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, medium_wall_holds)

In [59]:
create_json("sameclimber_mediumwall", path)

Path has been saved to results\sameclimber_mediumwall_climber_path.json.


In [10]:
create_json("sameclimber2_mediumwall", path)

Path has been saved to results\sameclimber2_mediumwall_climber_path.json.


### Smaller wall

In [11]:
# Define wall holds and goal state positions.
small_wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 8, 2) for y in range(1, 14, 2)
    ]

goal_state_positions = [np.array([7,13]), np.array([7,13])]

In [13]:
path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, small_wall_holds)

In [45]:
create_json("sameclimber_smallwall", path)

Path has been saved to results\sameclimber_smallwall_climber_path.json.


In [14]:
create_json("sameclimber2_smallwall", path)

Path has been saved to results\sameclimber2_smallwall_climber_path.json.


### Big wall

In [15]:
# Define wall holds and goal state positions.
big_wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 21, 3) for y in range(1, 26, 3)
    ]
big_wall_holds.append(Hold(3, 1))
big_wall_holds.append(Hold(5, 1))
big_wall_holds.append(Hold(5, 7))

goal_state_positions = [np.array([19,25]), np.array([19,25])]

In [16]:
path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, big_wall_holds)

In [95]:
create_json("sameclimber_bigwall", path)

Path has been saved to results\sameclimber_bigwall_climber_path.json.


In [17]:
create_json("sameclimber2_bigwall", path)

Path has been saved to results\sameclimber2_bigwall_climber_path.json.


### compare

In [18]:
def create_row(experiment_name, name, x_size, y_size, holds):
    file_path = f"results\{experiment_name}_climber_path.json"
    with open(file_path, "r") as file:
        path = json.load(file)
    new_row = {
        'name': name,
        'steps': len(path),
        'size': f"{x_size}x{y_size}",
        'number of holds': len(holds),
        'holds': sorted([(hold.x, hold.y) for hold in holds])
    }
    return pd.DataFrame([new_row])
    

#### Climber 1

In [68]:
df = pd.DataFrame(columns=['name', 'steps', 'size', 'number of holds', 'holds'])
df = pd.concat([df, create_row("2climbers_samewall", "easy wall", 10, 20, base_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber_mediumwall", "medium wall", 10, 20, medium_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber_hardwall", "hard wall", 10, 20, hard_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber_smallwall", "small wall", 8, 14, small_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber_bigwall", "big wall", 20, 25, big_wall_holds)], ignore_index=True)

In [70]:
# max display width 
pd.set_option('display.max_colwidth', None)
df

,name,steps,size,number of holds,holds
0,easy wall,17,10x20,50,"[(1, 1), (1, 3), (1, 5), (1, 7), (1, 9), (1, 11), (1, 13), (1, 15), (1, 17), (1, 19), (3, 1), (3, 3), (3, 5), (3, 7), (3, 9), (3, 11), (3, 13), (3, 15), (3, 17), (3, 19), (5, 1), (5, 3), (5, 5), (5, 7), (5, 9), (5, 11), (5, 13), (5, 15), (5, 17), (5, 19), (7, 1), (7, 3), (7, 5), (7, 7), (7, 9), (7, 11), (7, 13), (7, 15), (7, 17), (7, 19), (9, 1), (9, 3), (9, 5), (9, 7), (9, 9), (9, 11), (9, 13), (9, 15), (9, 17), (9, 19)]"
1,medium wall,19,10x20,39,"[(1, 1), (1, 4), (1, 7), (1, 10), (1, 13), (1, 16), (1, 19), (3, 1), (3, 1), (3, 4), (3, 7), (3, 10), (3, 13), (3, 16), (3, 19), (5, 1), (5, 1), (5, 4), (5, 7), (5, 7), (5, 10), (5, 13), (5, 16), (5, 19), (7, 1), (7, 4), (7, 7), (7, 10), (7, 13), (7, 16), (7, 19), (9, 1), (9, 4), (9, 7), (9, 10), (9, 13), (9, 16), (9, 19), (9, 19)]"
2,hard wall,22,10x20,25,"[(1, 1), (1, 4), (1, 7), (1, 10), (1, 13), (1, 16), (1, 19), (3, 1), (4, 1), (4, 4), (4, 7), (4, 10), (4, 13), (4, 16), (4, 19), (5, 1), (5, 7), (7, 1), (7, 4), (7, 7), (7, 10), (7, 13), (7, 16), (7, 19), (9, 19)]"
3,small wall,10,8x14,28,"[(1, 1), (1, 3), (1, 5), (1, 7), (1, 9), (1, 11), (1, 13), (3, 1), (3, 3), (3, 5), (3, 7), (3, 9), (3, 11), (3, 13), (5, 1), (5, 3), (5, 5), (5, 7), (5, 9), (5, 11), (5, 13), (7, 1), (7, 3), (7, 5), (7, 7), (7, 9), (7, 11), (7, 13)]"
4,big wall,27,20x25,66,"[(1, 1), (1, 4), (1, 7), (1, 10), (1, 13), (1, 16), (1, 19), (1, 22), (1, 25), (3, 1), (4, 1), (4, 4), (4, 7), (4, 10), (4, 13), (4, 16), (4, 19), (4, 22), (4, 25), (5, 1), (5, 7), (7, 1), (7, 4), (7, 7), (7, 10), (7, 13), (7, 16), (7, 19), (7, 22), (7, 25), (10, 1), (10, 4), (10, 7), (10, 10), (10, 13), (10, 16), (10, 19), (10, 22), (10, 25), (13, 1), (13, 4), (13, 7), (13, 10), (13, 13), (13, 16), (13, 19), (13, 22), (13, 25), (16, 1), (16, 4), (16, 7), (16, 10), (16, 13), (16, 16), (16, 19), (16, 22), (16, 25), (19, 1), (19, 4), (19, 7), (19, 10), (19, 13), (19, 16), (19, 19), (19, 22), (19, 25)]"


#### Climber 2

In [23]:
# max display width 
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(columns=['name', 'steps', 'size', 'number of holds', 'holds'])
df = pd.concat([df, create_row("2climbers_samewall(taller)", "easy wall", 10, 20, base_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber2_mediumwall", "medium wall", 10, 20, medium_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber2_hardwall", "hard wall", 10, 20, hard_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber2_smallwall", "small wall", 8, 14, small_wall_holds)], ignore_index=True)
df = pd.concat([df, create_row("sameclimber2_bigwall", "big wall", 20, 25, big_wall_holds)], ignore_index=True)
df

,name,steps,size,number of holds,holds
0,easy wall,16,10x20,50,"[(1, 1), (1, 3), (1, 5), (1, 7), (1, 9), (1, 11), (1, 13), (1, 15), (1, 17), (1, 19), (3, 1), (3, 3), (3, 5), (3, 7), (3, 9), (3, 11), (3, 13), (3, 15), (3, 17), (3, 19), (5, 1), (5, 3), (5, 5), (5, 7), (5, 9), (5, 11), (5, 13), (5, 15), (5, 17), (5, 19), (7, 1), (7, 3), (7, 5), (7, 7), (7, 9), (7, 11), (7, 13), (7, 15), (7, 17), (7, 19), (9, 1), (9, 3), (9, 5), (9, 7), (9, 9), (9, 11), (9, 13), (9, 15), (9, 17), (9, 19)]"
1,medium wall,17,10x20,39,"[(1, 1), (1, 4), (1, 7), (1, 10), (1, 13), (1, 16), (1, 19), (3, 1), (3, 1), (3, 4), (3, 7), (3, 10), (3, 13), (3, 16), (3, 19), (5, 1), (5, 1), (5, 4), (5, 7), (5, 7), (5, 10), (5, 13), (5, 16), (5, 19), (7, 1), (7, 4), (7, 7), (7, 10), (7, 13), (7, 16), (7, 19), (9, 1), (9, 4), (9, 7), (9, 10), (9, 13), (9, 16), (9, 19), (9, 19)]"
2,hard wall,19,10x20,25,"[(1, 1), (1, 4), (1, 7), (1, 10), (1, 13), (1, 16), (1, 19), (3, 1), (4, 1), (4, 4), (4, 7), (4, 10), (4, 13), (4, 16), (4, 19), (5, 1), (5, 7), (7, 1), (7, 4), (7, 7), (7, 10), (7, 13), (7, 16), (7, 19), (9, 19)]"
3,small wall,9,8x14,28,"[(1, 1), (1, 3), (1, 5), (1, 7), (1, 9), (1, 11), (1, 13), (3, 1), (3, 3), (3, 5), (3, 7), (3, 9), (3, 11), (3, 13), (5, 1), (5, 3), (5, 5), (5, 7), (5, 9), (5, 11), (5, 13), (7, 1), (7, 3), (7, 5), (7, 7), (7, 9), (7, 11), (7, 13)]"
4,big wall,25,20x25,66,"[(1, 1), (1, 4), (1, 7), (1, 10), (1, 13), (1, 16), (1, 19), (1, 22), (1, 25), (3, 1), (4, 1), (4, 4), (4, 7), (4, 10), (4, 13), (4, 16), (4, 19), (4, 22), (4, 25), (5, 1), (5, 7), (7, 1), (7, 4), (7, 7), (7, 10), (7, 13), (7, 16), (7, 19), (7, 22), (7, 25), (10, 1), (10, 4), (10, 7), (10, 10), (10, 13), (10, 16), (10, 19), (10, 22), (10, 25), (13, 1), (13, 4), (13, 7), (13, 10), (13, 13), (13, 16), (13, 19), (13, 22), (13, 25), (16, 1), (16, 4), (16, 7), (16, 10), (16, 13), (16, 16), (16, 19), (16, 22), (16, 25), (19, 1), (19, 4), (19, 7), (19, 10), (19, 13), (19, 16), (19, 19), (19, 22), (19, 25)]"


# Impossible Walls

In [7]:
# Define wall holds and goal state positions.
wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 11, 2) for y in range(1, 21, 9)
    ]

wall_holds.append(Hold(1, 7))
wall_holds.append(Hold(5, 7))

goal_state_positions = [np.array([9,19]), np.array([9,19])]

In [9]:
path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, wall_holds)

In [10]:
if path is None:
    print("No path found.")

No path found.


# Print the walls

P is starting point, H is hold, G is goal

In [64]:
def print_wall(x_size, y_size, holds_locations, x_goal, y_goal):
    wall_array = [['.' for _ in range(x_size)] for _ in range(y_size)]
    for pos in holds_locations:
        wall_array[pos[1]][pos[0]] = 'H'

    wall_array[7][1] = 'P'
    wall_array[1][3] = 'P'
    wall_array[1][5] = 'P'
    wall_array[7][5] = 'P'

    wall_array[y_goal][x_goal] = 'G'
    i = y_size - 1
    for y in range(y_size):
        print(f'{wall_array[y_size - y - 1]}     {i}')
        i -= 1

    to_print = []
    for x in range(x_size):
        to_print.append(str(x))
    print(to_print)


## Base Wall

In [43]:
x_size = 11
y_size = 21
goal_x = 9
goal_y = 19

### Easy

In [52]:
print_wall(x_size, y_size, [(hold.x, hold.y) for hold in base_wall_holds], goal_x, goal_y)

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     20
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'G', '.']     19
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     18
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     17
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     16
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     15
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     14
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     13
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     12
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     11
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     10
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     9
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     8
['.', 'P', '.', 'H', '.', 'P', '.', 'H', '.', 'H', '.']     7
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     6
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']    

### Medium

In [55]:
print_wall(x_size, y_size, [(hold.x, hold.y) for hold in medium_wall_holds], goal_x, goal_y)

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     20
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'G', '.']     19
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     18
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     17
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     16
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     15
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     14
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     13
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     12
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     11
['.', 'H', '.', 'H', '.', 'H', '.', 'H', '.', 'H', '.']     10
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     9
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     8
['.', 'P', '.', 'H', '.', 'P', '.', 'H', '.', 'H', '.']     7
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     6
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']    

### Hard

In [56]:
print_wall(x_size, y_size, [(hold.x, hold.y) for hold in hard_wall_holds], goal_x, goal_y)

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     20
['.', 'H', '.', '.', 'H', '.', '.', 'H', '.', 'G', '.']     19
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     18
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     17
['.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', '.']     16
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     15
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     14
['.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', '.']     13
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     12
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     11
['.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', '.']     10
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     9
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     8
['.', 'P', '.', '.', 'H', 'P', '.', 'H', '.', '.', '.']     7
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     6
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']    

## Small Wall

In [60]:
print_wall(8, 14, [(hold.x, hold.y) for hold in small_wall_holds], 7, 13)

['.', 'H', '.', 'H', '.', 'H', '.', 'G']     13
['.', '.', '.', '.', '.', '.', '.', '.']     12
['.', 'H', '.', 'H', '.', 'H', '.', 'H']     11
['.', '.', '.', '.', '.', '.', '.', '.']     10
['.', 'H', '.', 'H', '.', 'H', '.', 'H']     9
['.', '.', '.', '.', '.', '.', '.', '.']     8
['.', 'P', '.', 'H', '.', 'P', '.', 'H']     7
['.', '.', '.', '.', '.', '.', '.', '.']     6
['.', 'H', '.', 'H', '.', 'H', '.', 'H']     5
['.', '.', '.', '.', '.', '.', '.', '.']     4
['.', 'H', '.', 'H', '.', 'H', '.', 'H']     3
['.', '.', '.', '.', '.', '.', '.', '.']     2
['.', 'H', '.', 'P', '.', 'P', '.', 'H']     1
['.', '.', '.', '.', '.', '.', '.', '.']     0
['0', '1', '2', '3', '4', '5', '6', '7']


## Big Wall

In [65]:
print_wall(21, 26, [(hold.x, hold.y) for hold in big_wall_holds], 19, 25)

['.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'G', '.']     25
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     24
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     23
['.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.']     22
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     21
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     20
['.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.', '.', 'H', '.']     19
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']     18
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', 